In [1]:
# import os
# os.environ['TRANSFORMERS_CACHE'] = "/jagupard28/scr0/xiluo-speech/multi-quantizer-experiments/hf/checkpoints/"

from transformers import AutoModel
import torchaudio
from torchaudio.models.wav2vec2.utils import import_huggingface_model

hf_model = AutoModel.from_pretrained('facebook/hubert-large-ls960-ft').to('cuda') #TODO: change to hubert-large-ls960-ft
assert hf_model.__class__.__name__ in {"Wav2Vec2Model", "HubertModel"}

teacher_model = import_huggingface_model(hf_model).eval().to('cuda')

/nlp/scr/xiluo/miniconda3/envs/multiquant/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The model is not an instance of Wav2Vec2ForCTC. "lm_head" module is not imported.


In [2]:
from lhotse.recipes import download_librispeech, prepare_librispeech

# download_librispeech(dataset_parts="librispeech")
libri = prepare_librispeech(corpus_dir="./data/LibriSpeech/LibriSpeech", output_dir="./data/LibriSpeech/manifests")

Dataset parts:   0%|                                                                                         | 0/4 [00:00<?, ?it/s]
Distributing tasks: 0it [00:00, ?it/s]
                                      
Dataset parts:  25%|████████████████████▎                                                            | 1/4 [00:00<00:01,  2.45it/s]
Distributing tasks: 0it [00:00, ?it/s]
Distributing tasks: 43it [00:00, 306.82it/s]
Distributing tasks: 110it [00:00, 488.18it/s]
Distributing tasks: 162it [00:00, 388.91it/s]
Distributing tasks: 228it [00:00, 473.63it/s]
Distributing tasks: 280it [00:00, 396.90it/s]
Distributing tasks: 346it [00:00, 465.68it/s]
Distributing tasks: 412it [00:00, 518.04it/s]
Distributing tasks: 468it [00:01, 422.10it/s]
Distributing tasks: 534it [00:01, 478.68it/s]
                                             
Dataset parts:  50%|████████████████████████████████████████▌                                        | 2/4 [00:08<00:09,  4.79s/it]
Distributing tasks: 0it [00:

In [3]:
import torch

from lhotse import CutSet
from lhotse.dataset import BucketingSampler
from lhotse.dataset.input_strategies import AudioSamples
from torch.utils.data import DataLoader

class AudioSamplesDataset(torch.utils.data.Dataset):
    def __init__(self):
      self.collator = AudioSamples()

    def __getitem__(self, cuts: CutSet) -> dict:
        audio_padded, audio_lengths = self.collator(cuts)
        # print(cuts[0])
        cut_ids = [cuts[i].supervisions[0].id for i in range(len(cuts))]
        return { "audio_padded": audio_padded, "audio_lengths": audio_lengths, "cut_ids": cut_ids}

cuts_train = CutSet.from_manifests(**libri["train-clean-100"])

train_sampler = BucketingSampler(
    cuts_train,
    max_duration=60,
    shuffle=True,
    drop_last=True
)

train_loader = DataLoader(
    AudioSamplesDataset(),
    sampler=train_sampler,
    batch_size=None,
    num_workers=1
)

In [4]:
batch = next(iter(train_loader))

batch

{'audio_padded': tensor([[ 5.1880e-04,  5.1880e-04,  4.8828e-04,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [-6.1035e-05,  0.0000e+00,  9.1553e-05,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 3.2349e-03,  4.2419e-03,  4.7302e-03,  ..., -3.9673e-04,
          -6.4087e-04, -7.6294e-04]]),
 'audio_lengths': tensor([242640, 237840, 242880], dtype=torch.int32),
 'cut_ids': ['3879-174923-0028', '2289-152257-0023', '5463-39173-0008']}

In [5]:
# Which wav2vec 2/HuBERT transformer layer?
LAYER_OF_INTEREST=24
layer_index=LAYER_OF_INTEREST - 1

activations = []

# Register hook to trigger whenever forward() of nth layer is called
teacher_model.encoder.transformer.layers[layer_index].register_forward_hook(
    # Append outputs to list
    lambda teacher_model, inputs, outputs: activations.append(outputs.detach())
    # lambda teacher_model, inputs, outputs: print(outputs.shape)
  )

with torch.no_grad():
    final_activations, feat_lens=teacher_model(batch['audio_padded'].to('cuda'), batch["audio_lengths"].to('cuda'))
int_activations=activations[0]
activations.clear()
int_activations.shape

torch.Size([3, 758, 1024])

In [7]:
from multi_quantization import Quantizer
import torch

quantizer = Quantizer(
            dim=int_activations.shape[-1],
            num_codebooks=16,
            codebook_size=256,
        )

quantizer.load_state_dict(torch.load('./quantizers/full-layer24-16N-quantizer.pt'))
quantizer.to(torch.device('cuda'))

Quantizer(
  (to_logits): Linear(in_features=1024, out_features=4096, bias=True)
)

In [8]:
import pickle
import pandas as pd

In [9]:
from tqdm import tqdm

In [10]:
cuts_id_list = []
codebook_indexes_list = []

In [11]:
for batch in tqdm(train_loader):
    # embeddings = huber_model.extract_features(batch, layer=9)

    # codebook_indexes = quantizer.encode(encoder_embedding)
    # # [N, T, C]
    # codebook_indexes = codebook_indexes.to("cpu").numpy()
    
    # print(codebook_indexes)
    
    # for now save to csv
    # |  cut.cut_id  | codebook_indices |
    # | 123-456-123  | 31, 21, 30, 60   |
    with torch.no_grad():
        final_activations, feat_lens=teacher_model(batch['audio_padded'].to('cuda'), batch["audio_lengths"].to('cuda'))
    int_activations=activations[0]
    activations.clear()
    
    codebook_indexes = quantizer.encode(int_activations)
    # print(codebook_indexes.shape)
    codebook_indexes = codebook_indexes.to("cpu").numpy()
    # print(codebook_indexes)

    for i in range(codebook_indexes.shape[0]):
        cuts_id_list.append(batch['cut_ids'][i])
        codebook_indexes_list.append(codebook_indexes[i])

    

0it [00:00, ?it/s]/nlp/scr/xiluo/miniconda3/envs/multiquant/lib/python3.9/site-packages/multi_quantization/quantization.py:539: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  index=(this_indexes//saved_K).expand(*this_indexes.shape[:-1], dim)) +
7432it [1:18:19,  1.58it/s]


In [12]:
feats_df = pd.DataFrame({
    'cuts_id' : cuts_id_list,
    'codebook_indexes' : codebook_indexes_list
})

In [13]:
with open('./codebook_indexes/codebook_indexes_layer24_N16.pickle', 'wb') as handle:
    pickle.dump(feats_df, handle)

In [14]:
with open('./codebook_indexes/codebook_indexes_layer24_N16.pickle', 'rb') as handle:
    pickle_example = pickle.load(handle)

In [ ]:
pickle_example